In [129]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import requests
import re
import csv

# def extract(movie_name):
#     url = "https://www.google.com/search?q={}".format(movie_name)
#     page = requests.get(url)
#     soup = BeautifulSoup(page.content, "html.parser")

#     results = soup.find_all("div", class_="S8ee5.CwbYXd.wHYlTd")
#     print(results)

def extract_google_reviews(movie_name):
    # Initialize the Chrome web driver
    driver = webdriver.Chrome()

    # Search for the movie on Google
    driver.get("https://www.imdb.com/")
    search_box = driver.find_element(By.NAME, "q")
    search_box.send_keys(f"{movie_name}")
    search_box.send_keys(Keys.RETURN)

    # Click on the "All reviews" button to see more reviews
    home_url = driver.find_element(By.CLASS_NAME, "ipc-metadata-list-summary-item__t").get_attribute("href")
    home_url = home_url.split("?")[0]
    driver.get(f"{home_url}reviews?ref_=tt_urv")

    while True:
        try:
            button = driver.find_element(By.CLASS_NAME, "ipl-load-more__button")
            button.click()
            time.sleep(2)
        except:
            break
    
    rows = [] 

    complete_reviews = driver.find_elements(By.CLASS_NAME, "lister-item-content")
    # print(len(li))
    for complete_review in complete_reviews:
        date = complete_review.find_element(By.CLASS_NAME, "review-date").text
        title = complete_review.find_element(By.CLASS_NAME, "title").text
        try:
            rating = complete_review.find_element(By.CLASS_NAME, "rating-other-user-rating").text[:-3]
        except:
            rating = 0
        review = complete_review.find_element(By.CLASS_NAME, "content").text
        if review == "":
            continue

        match = re.search(r'.*?\d+ out of \d+ found', review, re.DOTALL)
        
        if match:
            pos = match.span()[0]
            review = review[:pos]

        review.replace("\n", " ")
        review.replace("\\'", "'")
        rows.append([movie_name, date, title, review, rating])
    with open('reviews.csv', 'a') as f:
        csvwriter = csv.writer(f, delimiter='~')
        csvwriter.writerows(rows)
    driver.quit()


In [130]:
fields = ['Movie Name', 'Date', 'Title', 'Review', 'Rating']
with open('reviews.csv', 'w') as f:
    csvwriter = csv.writer(f, delimiter='~')
    csvwriter.writerow(fields)
extract_google_reviews("Arjun Reddy")
    